In [15]:
import os
import ast
import openai
import time
from tqdm import tqdm
import json
from json import JSONDecodeError
import re

In [16]:
# openai.organization = ""  #My OpenAI account
# openai.api_key = "" #My Key
# openai.Model.list()

# # engines = openai.Engine.list()
# # print(engines)

<OpenAIObject list at 0x28443520b80> JSON: {
  "object": "list",
  "data": [
    {
      "id": "dall-e-3",
      "object": "model",
      "created": 1698785189,
      "owned_by": "system"
    },
    {
      "id": "whisper-1",
      "object": "model",
      "created": 1677532384,
      "owned_by": "openai-internal"
    },
    {
      "id": "davinci-002",
      "object": "model",
      "created": 1692634301,
      "owned_by": "system"
    },
    {
      "id": "dall-e-2",
      "object": "model",
      "created": 1698798177,
      "owned_by": "system"
    },
    {
      "id": "gpt-3.5-turbo-16k",
      "object": "model",
      "created": 1683758102,
      "owned_by": "openai-internal"
    },
    {
      "id": "tts-1-hd-1106",
      "object": "model",
      "created": 1699053533,
      "owned_by": "system"
    },
    {
      "id": "tts-1-hd",
      "object": "model",
      "created": 1699046015,
      "owned_by": "system"
    },
    {
      "id": "gpt-4",
      "object": "model",
      "cr

In [17]:
with open('llm_summary_by_gpt35.json', 'r') as f:
    data = json.load(f)
print(len(data))

1200


In [18]:
with open('evidence_corpus.json', 'r', encoding='utf-8') as f:
    evidence_data = json.load(f)
print(len(evidence_data))

8096


In [19]:
# Call GPT3.5 Model 
def get_completion_from_messages(messages, 
#                                  model= "gpt-3.5-turbo-16k-0613",
                                 model= "gpt-3.5-turbo-0125",
                                 temperature=0, 
                                 max_tokens=1024):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
        format={"type": "json_object"}
    )
    return response.choices[0].message["content"]

In [20]:
# delimiter_1 = "#"
# delimiter_2 = "##"

# system_message = f"""
# You are a Fact Checker.
# Given a news paragraph and evidence, assess the factual accuracy of the news story.
# Make a prediction and provide an explanation for your prediction based on the defined axes of scientific validity. Then score the news article and its equivalent evidence based on each axis of scientific validity between [-1, 1].

# Axes of scientific validity:
# 1. Alignment: Alignment is defined as news and evidence representing the same meaning about one scientific content.
# 2. Causation Confusion: The news article may confuse correlations presented in the scientific literature as causation. This could be one dimension in which the scientific validity is compromised.
# 3. Accuracy: This refers to how accurately the news item describes the scientific findings quantitatively and qualitatively.
# 4. Generalization: This refers to overgeneralization or oversimplification of the findings reported in the scientific literature.
# 5. Contextual Fidelity: Does the news article retain the broader context of the scientific finding?

# The user will input the news paragraph behind {delimiter_1} and the relevant evidence behind {delimiter_2}.
# """

In [21]:
delimiter_1 = "#"
delimiter_2 = "##"

system_message = f"""
You are a Fact Checker.
Given a news paragraph and evidence, assess the factual accuracy of the news story.
Make a prediction and provide an explanation for your prediction based on the defined axes of scientific validity. 

Axes of scientific validity:
1. Alignment: Alignment is defined as news and evidence representing the same meaning about one scientific content.
2. Causation Confusion: The news article may confuse correlations presented in the scientific literature as causation. This could be one dimension in which the scientific validity is compromised.
3. Accuracy: This refers to how accurately the news item describes the scientific findings quantitatively and qualitatively.
4. Generalization: This refers to overgeneralization or oversimplification of the findings reported in the scientific literature.
5. Contextual Fidelity: Does the news article retain the broader context of the scientific finding?

The user will input the news paragraph behind {delimiter_1} and the relevant evidence behind {delimiter_2}.
"""

In [22]:
# # The article discusses the challenges faced by organizations when transitioning
# for item in data[6:7]:
#     news = item['abstractive']
#     print(news,end='\n\n')

In [23]:
i = 0
generated_result = []
all_resp = []

In [26]:
for item in tqdm(data[62:]):
#     print(f"Predicting {i}")
    
    news = item['abstractive']
    #print(news)
    
    evidence_list = []
    if type(item['evidence'])==str:
        evidence_list.append(item['evidence'])
    else:
        evidence_list.extend(item['evidence'])
#     print(f"evidence list: {evidence_list}")
    
    evidence = [it['abstract'] for it in evidence_data if it['cord_uid'] in evidence_list]
    if evidence:
        evidence = evidence[0]
    else:
        evidence = ""
#     evidence = ''.join(evidence)

    user_message = f"""
     Here is one news paragraph {delimiter_1}{news} and relevant evidence {delimiter_2}{evidence}. 
     Determine if the evidence supports or refutes the news point of view. 
     Make a final prediction of 'True' if the news paragraph is factually true or 'False' if it is false based on the evidence under the keyword: 'prediction'. 
     Provide a comprehensive explanation for your prediction based on the defined axes of scientific validity under the keyword: 'reason'. 
     Then, score the news article based on each axis of scientific validity between [-1, 1] under the keyword: 'scores'. 
     For scoring, assign a float value in the range -1 and 1 to each axis, where -1 indicates strong disagreement, 0 indicates neutrality, and 1 indicates strong agreement.
     Provide the final output in JSON format containing the following three keys: prediction, reason, and scores.
    """
    
    assistant_message = """
    As a Fact Checker, carefully analyze the evidence abstracts and provide the result.
    Reference specific phrases or sentences from both the news story and the evidence to support your prediction.
    """
        
    messages =  [ {'role':'system', 'content': system_message},    
                  {'role':'user', 'content': user_message},  
                  {'role': 'assistant', 'content': assistant_message}]    
    
    response = get_completion_from_messages(messages)
#     print(response)
    
    result = dict()
    result['news'] = news
    result['evidence list'] = evidence_list
    result['ground truth'] = item['label']
    
    try:
        result.update(json.loads(response))
        
    except JSONDecodeError as e:
        pattern1 = r'"prediction": "(.*?)",\s*"reason": "(.*?)",\s*"scores": "(.*?)"'
        pattern2 = r"Prediction:\s*(True|False)\nReason:\s*(.*)\nReason:\s*(.*)"
        match1 = re.search(pattern1, response)
        match2 = re.search(pattern2, response)

        if match1:
            result['prediction'] = match1.group(1)
            result['reason'] = match1.group(2)
            result['scores'] = match1.group(3)
        
        elif match2:
            result['prediction'] = match2.group(1)
            result['reason'] = match2.group(2)
            result['scores'] = match1.group(3)
            
        else:
            pattern_prediction1 = r'##prediction\n(.*?)\n\n##reason'
            pattern_reason1 = r'##reason\n(.*?)\n\n##scores'
            pattern_scores1 = r'##scores\n(.*?)$'
            
            prediction_match = re.search(pattern_prediction1, response, re.DOTALL)
            reason_match = re.search(pattern_reason1, response, re.DOTALL)
            scores_match = re.search(pattern_scores1, response, re.DOTALL)

            # Extracting information
            if prediction_match and reason_match and scores_match:
                result['prediction'] = re.search(pattern_prediction1, response, re.DOTALL).group(1).strip()
                result['reason'] = re.search(pattern_reason1, response, re.DOTALL).group(1).strip()
                result['scores'] = re.search(pattern_scores1, response, re.DOTALL).group(1).strip()

            else:
                prediction_pattern = r'##prediction##\n(.*?)\n\n'
                reason_pattern = r'##reason##\n(.*?)\n\n'
                scores_pattern = r'##scores##\n(.*?)$'

                prediction_match = re.search(prediction_pattern, response, re.DOTALL)
                reason_match = re.search(reason_pattern, response, re.DOTALL)
                scores_match = re.search(scores_pattern, response, re.DOTALL)

                if prediction_match and reason_match and scores_match:
                    result['prediction'] = prediction_match.group(1)
                    result['reason'] = reason_match.group(1)
                    result['scores'] = scores_match.group(1)


    all_resp.append(response)
    generated_result.append(result)
    
    time.sleep(2)
    i += 1
    
#     break

  0%|          | 5/1138 [00:55<3:30:37, 11.15s/it]


APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [145]:
type(response)

str

In [25]:
i

62

In [137]:
re.search(scores_pattern, response, re.DOTALL)

<re.Match object; span=(1632, 1731), match='##scores##\nAlignment: 1\nCausation Confusion: 1\>

In [138]:
re.search(pattern_prediction, response, re.DOTALL)

In [151]:
# Pattern matching
# pattern_prediction = r'##prediction\n(.*?)\n\n##reason'
# pattern_reason = r'##reason\n(.*?)\n\n##scores'
# pattern_scores = r'##scores\n(.*?)$'

# # Extracting information
# prediction = re.search(pattern_prediction, response, re.DOTALL).group(1).strip()
# reason = re.search(pattern_reason, response, re.DOTALL).group(1).strip()
# scores = re.search(pattern_scores, response, re.DOTALL).group(1).strip()

In [119]:
scores

'Alignment: 1\nCausation Confusion: 1\nAccuracy: 1\nGeneralization: 1\nContextual Fidelity: 1'

In [125]:
# results = dict()
# results.update(json.loads(response))

In [126]:
# pattern1 = r'"prediction": "(.*?)",\s*"reason": "(.*?)",\s*"scores": "(.*?)"'
# pattern2 = r"Prediction:\s*(True|False)\nReason:\s*(.*)\nReason:\s*(.*)"
# match1 = re.search(pattern1, response)
# match2 = re.search(pattern2, response)

# if match1:
#     print('match1')
# #     result['prediction'] = match1.group(1)
# #     result['reason'] = match1.group(2)
# #     result['scores'] = match1.group(3)

# elif match2:
#     print('match2')
# #     result['prediction'] = match2.group(1)
# #     result['reason'] = match2.group(2)
# #     result['scores'] = match1.group(3)

In [147]:
result

{'news': "The article discusses the challenges faced by organizations when transitioning to agile delivery methods. While the tools and techniques themselves are not difficult to learn, the main issues arise from the interface between the agile teams and the organization's existing norms and practices. To address these challenges, a modified Activity Theory framework was applied to a case study in a public sector organization. This framework helped identify obstacles and issues in the learning and development process. The study emphasizes the importance of understanding the historical, cultural, and behavioral aspects that impact the transition to agile delivery methods.",
 'evidence list': ['06cj16bv', '06cj16bv', 'd7ux4e5r'],
 'ground truth': 1,
 'prediction': 'False',
 'reason': "1. Alignment: The evidence aligns with the news article in terms of discussing the challenges faced by organizations when transitioning to agile delivery methods and the application of a modified Activity T

In [148]:
# prediction_start = response.find('"prediction": "') + len('"prediction": "')
# prediction_end = response.find('"', prediction_start)
# reason_start = response.find('"reason": "') + len('"reason": "')
# reason_end = response.find('"', reason_start)
# scores_start = response.find('\n    "scores": {\n') + len('\n    "scores": {\n')
# scores_end = response.find('}', scores_start)

# # Extract the prediction and reason values
# results['prediction'] = response[prediction_start:prediction_end]
# results['reason'] = response[reason_start:reason_end]
# results['scores'] = {response[scores_start:scores_end]}
# print(results)

In [149]:
result['scores']

'Alignment: 1\nCausation Confusion: 0\nAccuracy: 1\nGeneralization: 0\nContextual Fidelity: 1'

In [63]:
print(result['scores'])

{'        "Alignment": 1,\n        "Causation Confusion": 1,\n        "Accuracy": 1,\n        "Generalization": 1,\n        "Contextual Fidelity": 1\n    '}


In [42]:
result['scores']

{'        "Alignment": 1,\n        "Causation Confusion": 0,\n        "Accuracy": 1,\n        "Generalization": 0,\n        "Contextual Fidelity": 1\n    '}

In [36]:
response.find('\n    "scores": {\n') + len('\n    "scores": {\n')

716

In [39]:
response[scores_start:scores_end]

'        "Alignment": 1,\n        "Causation Confusion": 0,\n        "Accuracy": 1,\n        "Generalization": 0,\n        "Contextual Fidelity": 1\n    '

In [ ]:
generated_result[3]

In [11]:
# generated_result.remove(generated_result[-1])

In [82]:
with open('explanation_cot_prediction_GPT35_llm.json', 'w') as file:
    json.dump(generated_result, file)

In [95]:
type(generated_result)

list

In [13]:
for i in generated_result:
    if 'prediction' in i.keys():
        continue
    else:
        print(i, end='\n\n\n')
#     print(type(i))
#     break

In [98]:
# generated_result[56]['prediction'] = 'True'
# generated_result[56]['reason'] = "The evidence provided aligns with the news paragraph and supports the claim made. The patient was initially admitted for chronic obstructive pulmonary disease exacerbation secondary to pneumonia but was later evaluated for COVID-19 due to a rapidly progressing course of events. The news paragraph accurately summarizes the key findings from the evidence without overgeneralizing or oversimplifying the information."

In [99]:
# generated_result[56]

In [100]:
type(response)

str

In [101]:
print(response)

{
  "prediction": "True",
  "reason": "The evidence supports the central claim of the news paragraph that Human bocavirus (HBoV) has been identified as the cause of viral respiratory infections in children. The evidence states that HBoV was identified in respiratory samples from children suffering from viral respiratory infections. It also mentions that a large number of clinical studies have been performed to address the clinical significance of HBoV detection and the molecular biology of the virus."
}


In [102]:
pattern1 = r'"prediction": "(.*?)",\s*"reason": "(.*?)"'
pattern2 = r"Prediction:\s*(True|False)\nReason:\s*(.*)"
match1 = re.search(pattern1, response)
match2 = re.search(pattern2, response)

if match1:
    result['prediction'] = match1.group(1)
    result['reason'] = match1.group(2)
#     print(dict({'prediction':result['prediction'], 'reason':result['reason']}))
elif match2:
    result['prediction'] = match2.group(1)
    result['reason'] = match2.group(2)
#     print(dict({'prediction':result['prediction'], 'reason':result['reason']}))

In [103]:
result

{'news': 'Human bocavirus (HBoV) has been identified as the cause of viral respiratory infections in children, according to a groundbreaking study published in the journal Nature Medicine. HBoV, discovered in 2005, is the second human parvovirus with pathogenic potential. Although difficult to grow in cell culture, HBoV is suspected to be a pathogen. Further research is needed to fully understand its role in viral respiratory infections.',
 'evidence list': ['1qyuhbz0', '1qyuhbz0', '8y7218oj'],
 'ground truth': 1,
 'prediction': 'True',
 'reason': 'The evidence supports the central claim of the news paragraph that Human bocavirus (HBoV) has been identified as the cause of viral respiratory infections in children. The evidence states that HBoV was identified in respiratory samples from children suffering from viral respiratory infections. It also mentions that a large number of clinical studies have been performed to address the clinical significance of HBoV detection and the molecular 